In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score

In [2]:
df = pd.read_csv(r'/root/autodl-tmp/CommitFit/dataset/multi-lang-exp/dataset.csv', index_col=0, encoding='utf_8_sig')
df.fillna('', inplace=True)
label2id={'A':0, 'C':1, 'P':2}
df = df.rename(columns={'maintenance_type':'labels','commit_message':'text','commit_diff':'diffs'})
df = df.replace({"labels": label2id})
# # print(df)
df
# test_sample = df.sample(n=3, random_state=1)

/tmp/ipykernel_986/1594221776.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"labels": label2id})


,language,commit,commit_url,labels,user,repo,feature,diffs,msgs
0,C++,be937a3290223f926fe50684f1344569a573ed4b,https://github.com/tensorflow/tensorflow/commi...,1,tensorflow,tensorflow,"[1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 6, ...",diff --git a/tensorflow/compiler/mlir/lite/fla...,Fixing per axis quantization bug in flatbuffer...
1,C++,c800d2e36954edddcb83aa1df7f623f2780c7780,https://github.com/tensorflow/tensorflow/commi...,0,tensorflow,tensorflow,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/tensorflow/python/tools/saved_mod...,Add support for python expression as SavedMode...
2,C++,469e56eeff17eb857c95e935ca7b49723c43470e,https://github.com/tensorflow/tensorflow/commi...,2,tensorflow,tensorflow,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/.github/ISSUE_TEMPLATE/00-bug-iss...,Fix GitHub issue templates. Actual fix for #36...
3,C++,24f5d302c0906f3377f4d6bcc2735d943ce13e3d,https://github.com/tensorflow/tensorflow/commi...,0,tensorflow,tensorflow,"[3, 37, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/tensorflow/compiler/xla/pjrt/dist...,"Add new PjRT distributed APIs (KeyValueDelete,..."
4,C++,1effdb7b959503596b07f7f3e74618ab63f9e5e5,https://github.com/tensorflow/tensorflow/commi...,2,tensorflow,tensorflow,"[1, 0, 12, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/tensorflow/core/common_runtime/ea...,Remove IsLocalDevice() since it's no longer us...
...,...,...,...,...,...,...,...,...,...
1576,Java,e4bbfd537a63a52a6094491504fc7b0bb54bcb0e,,1,kiegroup,drools,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/drools-persistence-jpa/src/main/j...,fixing reflection constructor to use int inste...
1577,Java,75b81dcbd26a11d5b370ea5bdf102c2d62ebd0c3,,1,kiegroup,drools,"[2, 6, 2, 3, 0, 0, 0, 1, 7, 0, 0, 0, 0, 0, 0, ...",diff --git a/drools-compiler/src/main/java/org...,Fix OutOfBoundException on- MemoryFileSystem.e...
1578,Java,6bc9677bbe919cd5bcadd8af2f5b0c838757a4ce,,1,kiegroup,drools,"[1, 1, 5, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/drools-core/src/main/java/org/dro...,[DROOLS-293] fix ObjectTypeNode id creation an...
1579,Java,48701edad8513b27acec7216581e64637157c86a,,0,kiegroup,drools,"[2, 24, 2, 4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/drools-core/src/main/java/org/dro...,DROOLS-515 Kie-Camel is broken after Camel Upd...


In [3]:
train, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test = train_test_split(temp_df, test_size=0.5, random_state=42)

In [4]:
train = train.groupby("labels", group_keys=False).apply(lambda x: x.sample(n=20, random_state=42))

/tmp/ipykernel_986/2035981014.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train = train.groupby("labels", group_keys=False).apply(lambda x: x.sample(n=10, random_state=42))


In [5]:
# 假设你有一个DataFrame df，其中包含'commit_message'和'label'列
# df = pd.DataFrame({'commit_message': [...], 'label': [...]})

# 1. 数据预处理和特征提取
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train['diffs'])
y_train = train['labels']

# 3. 训练XGBoost模型
clf = XGBClassifier()
clf.fit(X_train, y_train)

# 4. 评估模型
y_pred = clf.predict(vectorizer.transform(test['diffs']))
print("Accuracy:", accuracy_score(test['labels'], y_pred))
print("Classification Report:\n", classification_report(test['labels'], y_pred,digits=4))


Accuracy: 0.4957983193277311
Classification Report:
               precision    recall  f1-score   support

           0     0.2909    0.5000    0.3678        32
           1     0.3137    0.6275    0.4183        51
           2     0.8642    0.4516    0.5932       155

    accuracy                         0.4958       238
   macro avg     0.4896    0.5264    0.4598       238
weighted avg     0.6692    0.4958    0.5254       238

